# Translate the AM Bench metadata in Excel tables to XML documents

The goal of this notebook is to generate XML documents for metadata describing AM Bench 2022 series samples, build processes and measurements entered in excel spreadsheets and optionally to upload them in <b>private</b> CDCS AM Bench project database. 

The persistent ID (<b>PID</b>) to an XML document is assigned by CDCS for its first time being uploaded in CDCS. Afterwards, the PID of the document remains the same even if its content changes. Therefore, generating XML documents requires to query the CDCS AM Bench database in order to check whether XML documents already exist in the CDCS or not. The PIDs of the existing documents are assigned to the updated ones.

Any user can query the public CDCS AM Bench database as anonymous user. Therefore, anyone can generate XML documents. 
For uploading documents in private CDCS database you need a permission in order to do so. If you wish to get the permission, please contact Lyle E. Levine (lyle.levine@nist.gov).

In [1]:
import sys
import os
import io
import pprint
import lxml.etree as ET
import xml.dom.minidom
import importlib
import glob
import json
import string
import datetime
import pandas
import getpass
import matplotlib.pyplot as plt
import requests
from PIL import Image, ImageDraw, ImageFont

In [3]:
# Set False for requiring to validate XML document when generate them.
# Instead validate them after all documents are generated.

import pyxb
pyxb.RequireValidWhenGenerating(False);

# Initiantiate __CONFIG class 
- In order to run this notebook create your own configuration file in JSON format. Please see the example given in default_config.json.
- Enter your json file in the argument of the constructor of <code>__CONFIG class</code> defined in config.py. If no argument is passed in the constructor, default_config.json is used.


In [4]:
# Import config and instantiate __CONFIG class.
import config
from config import __CONFIG

CONFIG = __CONFIG(conf_json = "./myconfig_linux.json")


In [13]:
# If USER or PASS are null in your configuration setting, you are asked enter them in the prompts interactively.
# For anonymous user enter nothing in the prompts you get when you run this cell.

if CONFIG.USER is None:
    CONFIG.USER = input('username: ')
if CONFIG.PASS is None:
    CONFIG.PASS = getpass.getpass('enter password ')

AUTH=(CONFIG.USER, CONFIG.PASS)    


In [6]:
# Include the directory path for the required Python modules.

sys.path.insert(0, CONFIG.pyUTILS_path)
import ambench.cdcs_utils
from ambench.cdcs_utils import AMBench2022, xmlschema
from ambench.mapping import new_mapper


# Create AMBench2022 instance
* AMBench2022 is a wrapper class of which base class is CDCS from pycdcs. It has additional methods including querying, and uploading XML schemas and documents in the CDCS instance.

In [7]:
ambench2022=AMBench2022(CONFIG.TEMPLATE,CONFIG.AMBENCH_URL,auth=AUTH)

# Create XML Validator from in-memory schema files using xmlschema.

In [8]:
# xsd_filename is a file path of ROOT_SCHEMA

xsd_filename=f'{CONFIG.XSD}{CONFIG.ROOT_SCHEMA}'
VALIDATOR=xmlschema.XMLSchema(xsd_filename,build=False)
VALIDATOR.build()
VALIDATOR.validity 

'valid'

# Generate, Validate and Load XML files
* Use function <code>map_them</code> to map the metadata for samples or measurements from AM Bench project entered in Excel spreadsheets to XML files.  
    * Select the types of samples or measurements to map and pass them as argment <code>DOC_TYPES2LOAD</code> of <code>map_them</code>.
    * There are dependencies between the XML documents because of hyper links referring from one document to another. For example, documents for measurements refer to the documents describing the specimens used in their measurements. Or a specimen document may refer to the one for a source build part, or a material and etc.  Therefore, the documents must be generated in a proper order by their types as given in <code>ALL_DOC_TYPES</code> below.
* Validate XML documents using <code>validate_amdocs</code>.
* Upload using <code>load_amdocs_cdcs</code>

In [9]:
def validate_amdocs(amdocs):
    '''
    Validate generated XML files.
    
    amdocs: a dict of generated  XML files and and flag indicating 
            whether XML file is new or not.
    '''
    for xmlfile,is_new in amdocs.items():
        v=VALIDATOR.is_valid(xmlfile)
        if not(v):
            try:
                VALIDATOR.validate(xmlfile)
            except Exception as e:
                print(xmlfile,"\n",e,"\n=====\n")
                raise e
    return True

In [10]:
def load_amdocs_cdcs(amdocs):
    '''
    Load XML documents to CDCS.
    
    amdocs: a dict of XML files and their flags indicating whether each XML file is new or not.
    '''
    uploaded={}
    errors={}
    for f,is_new in amdocs.items():
        fn=os.path.basename(f)
        if is_new:
            print('upload new:', f)
            response=ambench2022.upload_data(f)
        else:
            print('update existing:', f)
            response=ambench2022.update_data(f)

        if response.ok:
            uploaded[fn]=response.json()    
        else:
            errors[fn]=response.json()
    return uploaded,errors

In [11]:
# The order of loading doc types.
ALL_DOC_TYPES=['Material','AMPowder','AMBuildPlate','AMBuildPart','AMBSpecimen',
               'AMComposition', 'AMLaserAbsorptivity', 'AMRadiography',  'AMMechanicalTesting']

def map_them(DOC_TYPES2LOAD,doLoad=False,breakOnError=True,folder='TEMP'):
    '''
    Generate, and validate XML files.  If doLoad is True upload them to the CDCS AM Bench database. 
    
    DOC_TYPES2LOAD: List of document types to generate and upload 
    doLoad: Flag whether to upload the generated documents to the CDCS instance or not.
    breakOnError: Flag whether to break the loop or not if there is an error in generating a document.
    folder: Name of folder where to stage the generated documents in local file system. The full folder 
            path depends on document type <outfolder> below.
    '''
    all_handled={}
    for DOC_TYPE in DOC_TYPES2LOAD:
        print("DOC_TYPE:",DOC_TYPE)
        
        outfolder=f"{CONFIG.LOADING}/{folder}/{DOC_TYPE}/"
        os.makedirs(outfolder,exist_ok=True)
        
        MAPPER=new_mapper(ambench2022, DOC_TYPE, CONFIG)
        try :
            amdocs=MAPPER.map_from_excel(outfolder,verbose=True) 
            ok = validate_amdocs(amdocs)
            if not ok:
                print()
            if doLoad and ok:
                load_amdocs_cdcs(amdocs)
            all_handled[DOC_TYPE]=amdocs
        except Exception as e:
            print(e)
            if breakOnError:
                break
    return all_handled

In [12]:
%%time
# Specify the list of AM Bench documnent types to map
doc_types=ALL_DOC_TYPES

r=map_them(doc_types,doLoad=False)

DOC_TYPE: Material
Found: TI64-D1  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/PBBOBFU60S4ZJO5D update doc from excel
Found: AA5182-D1  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/ULQF8WE9O3Q7DJKP update doc from excel
Found: 718-SH1  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/P7TVXK25GCW4ZWDS update doc from excel
DOC_TYPE: AMPowder


 83%|████████▎ | 10/12 [00:00<?, ?it/s]

Found: K201801  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/526BWB5BAIPV8FVB update doc from excel
Found: M051701  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/Z9C1W8ZDI2FSVEKB update doc from excel
Found: M341901  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/T5E99OQBKI2VNFUC update doc from excel
Found: M421601  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/L3GB36AAKOEK8QNU update doc from excel
DOC_TYPE: AMBuildPlate


100%|██████████| 31/31 [00:00<00:00, 49.63it/s]


Found: AMB2022-718-AMMT-B1  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/N5C1HKQ9BZO3ZS66 update doc from excel
Found: AMB2022-718-AMMT-B2  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/T8FETDWTHOBMD3LM update doc from excel
Found: AMB2022-718-AMMT-B3  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/J3Q32LE63WH09S57 update doc from excel
Found: AMB2022-718-AMMT-B4  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/7O3WK5HX4C3LOSTA update doc from excel
Found: AMB2022-718-AMMT-B5  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/UNQ4PM46IB11Y04O update doc from excel
Found: AMB2022-718-AMMT-B6  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/GK1GVW851DEFIW2Y update doc from excel
Found: AMB2022-718-AMMT-B7  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/RRWX582XWX7TLZCC update doc from excel
Found: AMB2022-718-AMMT-B8  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/L3K0CEHHHWE2SDKL update doc from excel
Found: AMB2022-718-AMMT-B9  ==> 

...

100%|██████████| 100/100 [00:01<00:00, 49.88it/s]


Reading excel  3.085355043411255


100%|██████████| 12/12 [00:00<00:00, 15.32it/s]


Upload pids  6.188572645187378
Found: AMB2022_Mech-Testing_44519001  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/8AJQPZXAX2MD83YA update doc from excel
Found: AMB2022_Mech-Testing_44519002  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/9KG30PFO133OE10N update doc from excel
Found: AMB2022_Mech-Testing_44519003  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/GBLHRU6WNCHG65R8 update doc from excel
Found: AMB2022_Mech-Testing_44522001  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/RJ6FMA4GC5EDRSSA update doc from excel
Found: AMB2022_Mech-Testing_44522002  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/JVOD3OUUBPYC8ZO9 update doc from excel
Found: AMB2022_Mech-Testing_44522003  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/GXK73SNYHQFRG5F2 update doc from excel
Found: AMB2022_Mech-Testing_625-CBM-B1-TH1-F  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/LT0F73ZNQW0QLBKX update doc from excel
Found: AMB2022_Mech-Testing_625-CBM-B2-0-X.0

Found: AMB2022_Mech-Testing_625-CBM-B2-60-X.12-3  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/KJL5QHQPUL60MV9D update doc from excel
Found: AMB2022_Mech-Testing_625-CBM-B2-60-X.12-4  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/P3BLWKEM5FVRWUDK update doc from excel
Found: AMB2022_Mech-Testing_625-CBM-B2-60-X.14-1  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/QECRJGY9QGPNFW0J update doc from excel
Found: AMB2022_Mech-Testing_625-CBM-B2-60-X.14-2  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/AYTTGF50B1TG3KDQ update doc from excel
Found: AMB2022_Mech-Testing_625-CBM-B2-60-X.14-3  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/VX9GILX5G8UEPLMS update doc from excel
Found: AMB2022_Mech-Testing_625-CBM-B2-60-X.14-4  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/YR59WU8M9KIHRUW2 update doc from excel
Found: AMB2022_Mech-Testing_625-CBM-B2-60-XY.04-1  ==>  https://ambench2022.nist.gov/pid/rest/local/cdcs/085206XQT3QAA8MO update doc from exce